In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import plotly.express as px
import plotly.io as pio
import timesynth as ts
from scipy.special.cython_special import pseudo_huber

from src.synthetic_ts.autoregressive import AutoRegressive

pio.templates.default = 'plotly_white'
np.random.seed()


In [2]:
SAVE_FIGURES = False
if SAVE_FIGURES:
    os.makedirs('img/chapter_01', exist_ok=True)

# Data Generating Process and Synthetic Time Series

In [3]:
def plot_time_series(time, values, label, legends=None):
    if legends is not None:
        assert len(legends) == len(values)
    if isinstance(values, list):
        series_dict = {'Time': time}
        for v, l in zip(values, legends):
            series_dict[l] = v
        plot_df = pd.DataFrame(series_dict)
        plot_df = pd.melt(plot_df, id_vars='Time', var_name='ts', value_name='Value')
    else:
        series_dict = {'Time': time, 'Value': values, 'ts': ''}
        plot_df = pd.DataFrame(series_dict)

    if isinstance(values, list):
        fig = px.line(plot_df, x='Time', y='Value', line_dash='ts')
    else:
        fig = px.line(plot_df, x='Time', y='Value')

    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title={
            'text': label,
            'y': 0.9,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {
                'size': 25
            }
        },
        yaxis=dict(
            title={
                'text': 'Value',
                'font': dict(size=12)
            }
        ),
        xaxis=dict(
            title={
                'text': 'Time',
                'font': dict(size=12)
            }
        )
    )
    return fig

def generate_timeseries(signal, noise=None):
    time_sampler = ts.TimeSampler(stop_time=20)
    regular_time_samples = time_sampler.sample_regular_time(num_points=100)
    timeseries = ts.TimeSeries(signal_generator=signal, noise_generator=noise)
    samples, signals, errors = timeseries.sample(regular_time_samples)
    return samples, regular_time_samples, signals, errors

## White Noise

In [4]:
# Generate the time axis with sequential numbers up to 200
time = np.arange(200)

# Sample 200 random values
values = np.random.randn(200) * 100
fig = plot_time_series(time, values, '')
if SAVE_FIGURES:
    fig.write_image('img/chapter_01/white_noise_process.png')
fig.show()

## Red Noise

In [5]:
# Setting the correlation coefficient
r = 0.4

# Generate time axis
time = np.arange(200)

# Generate white noise
white_noise = np.random.randn(200) * 100

# Create red noise by introducing correlation between subsequent values in the white noise
values = np.zeros(200)
for i, v in enumerate(white_noise):
    if i == 0:
        values[i] = v
    else:
        values[i] = r*values[i-1] + np.sqrt((1-np.power(r, 2))) * v

fig = plot_time_series(time, values, '')
if SAVE_FIGURES:
    fig.write_image('img/chapter_01/red_noise_process.png')
fig.show()

## Sinusoidal

In [6]:
# Sinusoidal signal with aplitude=1.5 & frequency=0.25
signal_1 = ts.signals.Sinusoidal(amplitude=1.5, frequency=0.25)
# Sinusoidal signal with amplitude=1 & frequency=0.5
signal_2 = ts.signals.Sinusoidal(amplitude=1, frequency=0.5)

# Generate time series
samples_1, regular_time_samples, sginals_1, errors_1 = generate_timeseries(signal=signal_1)
samples_2, regular_time_samples, signals_2, errors_2 = generate_timeseries(signal=signal_2)

In [7]:
fig = plot_time_series(regular_time_samples, [samples_1, samples_2], '',
                       legends=['Amplitude = 1.5 | Frequency = 0.25', 'Amplitude = 1 | Frequency = 0.5'])

if SAVE_FIGURES:
    fig.write_image('img/chapter_01/sinusoidal_waves.png')
fig.show()

## Psuedo Periodic

In [8]:
signal = ts.signals.PseudoPeriodic(amplitude=1, frequency=0.25)
samples, regular_time_samples, signals, errors = generate_timeseries(signal=signal)

In [9]:
fig = plot_time_series(regular_time_samples, samples, '')
if SAVE_FIGURES:
    fig.write_image("imgs/chapter_1/pseudo_process.png")
fig.show()

## Auto Regressive

In [10]:
from src.synthetic_ts.autoregressive import AutoRegressive

signal = AutoRegressive(ar_param=[1.5, -0.75])

samples, regular_time_samples, signals, errors = generate_timeseries(signal=signal)

In [11]:
fig = plot_time_series(regular_time_samples, samples, '')

if SAVE_FIGURES:
    fig.write_image('img/chapter_01/autoregressive_process.png')
fig.show()

## Mix and Match

In [12]:
# Use Psuedo Periodic  to combine White Noise with Auto Regression signal
psuedo_samples, regular_time_samples, _, _ = generate_timeseries(signal=ts.signals.PseudoPeriodic(amplitude=1, frequency=0.25), noise=ts.noise.GaussianNoise(std=0.3))

# Generate an autoregressive signal
ar_samples, regular_time_samples, _, _ = generate_timeseries(signal=AutoRegressive(ar_param=[1.5, -0.75]))

# Combine the two signals
timeseries_ = psuedo_samples * 2 + ar_samples

In [13]:
fig = plot_time_series(regular_time_samples, timeseries_, '')
if SAVE_FIGURES:
    fig.write_image('img/chapter_01/mix_and_match.png')
fig.show()

## Non-Stationary: Sinusoidal with Trend and White Noise

In [15]:
# Create trend and seasonality
signal = ts.signals.Sinusoidal(amplitude=1, frequency=0.25)

# White noise with standard deviation = 0.3
noise = ts.noise.GaussianNoise(std=0.3)

# Generate the time series
sinusoidal_samples, regular_time_samples, _, _ =generate_timeseries(signal=signal, noise=noise)

# Regular_time_samples is a linear increasing time axis and can be used as a trend
trend = regular_time_samples * 0.4

# Combining the signal and trend
timeseries_ = sinusoidal_samples + trend

In [16]:
fig = plot_time_series(regular_time_samples, timeseries_, '')

if SAVE_FIGURES:
    fig.write_image('img/chapter_01/trend.png')
fig.show()

## Non-Stationary: Sinusoidal and Time Varying Noise

In [17]:
sinusoidal_samples, regular_time_samples, _, _ = generate_timeseries(signal=ts.signals.Sinusoidal(amplitude=1, frequency=0.25))

In [18]:
noise = [np.random.randn() * np.sqrt(i) for i, v, in enumerate(regular_time_samples)]

In [19]:
fig = plot_time_series(regular_time_samples, sinusoidal_samples + noise, '')

if SAVE_FIGURES:
    fig.write_image('img/chapter_01/non_stationary_sinusoidal.png')
fig.show()